In [1]:
# from google.colab import drive
# drive.mount('/gdrive',force_remount=True)
# %cd /gdrive
# %cd MyDrive
# # %cd Colab Notebooks
# %cd 'bodyshop repair'
# path = "bodyshop_repair_tensorflow/working_notebook/data.json"

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('D:\OneDrive - NITT\Custom_Download\spell_corrected_final.json')

In [3]:
# Splitting dataset into train, test and dev
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.4, random_state=42)
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

In [4]:
from tqdm.notebook import tqdm

In [5]:
train_corpus = []
for text in tqdm(train_df['Clean_Damage']):
  train_corpus.append(text)

test_corpus = []
for text in tqdm(test_df['Clean_Damage']):
  test_corpus.append(text)

  0%|          | 0/255349 [00:00<?, ?it/s]

  0%|          | 0/85116 [00:00<?, ?it/s]

In [6]:
# import the library
import gensim

# read the corpus and convert it to tagged document.
def read_corpus(corpus):
    i = 0
    for doc_fname in corpus:
        tokens = doc_fname.split()
        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
        i = i + 1

In [7]:
# get the tagged corpus
train_corpus = list(read_corpus(train_corpus))

test_corpus = list(read_corpus(test_corpus))

In [8]:
# first tagged document
train_corpus[0]

TaggedDocument(words=['entire', 'front', 'end', 'down', 'the', 'drivers', 'side'], tags=[0])

In [9]:
# setup doc2vec
model = gensim.models.doc2vec.Doc2Vec(vector_size=100)

In [10]:
# build the vocabulary
model.build_vocab(train_corpus)

In [11]:
# train the doc2vec on the given corpus
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [12]:
# get the doc2vec embedding of the first document.
model.infer_vector(train_corpus[0][0])

array([-0.03418214, -0.04329872, -0.04091361, -0.0268693 , -0.0666113 ,
        0.09817664, -0.04009007, -0.0674959 ,  0.00294965, -0.02260552,
        0.02754609,  0.039796  ,  0.08810495,  0.01184182, -0.01478468,
        0.09585541, -0.02739911,  0.08738523,  0.01726268,  0.09892351,
       -0.09619522, -0.05445495, -0.04984544,  0.00690356,  0.09945501,
        0.01588379,  0.05130263,  0.03701887, -0.00760381, -0.02255135,
       -0.03359864,  0.04904255, -0.02870908,  0.07469813,  0.095943  ,
       -0.12951012,  0.02155304,  0.04695604, -0.03179697,  0.04059188,
        0.02558925,  0.08457031, -0.1444151 ,  0.03344148,  0.00905937,
        0.06960191,  0.00882101, -0.03214044,  0.0105387 ,  0.01212643,
       -0.04520219,  0.05860241, -0.00079776,  0.03779958,  0.11117833,
        0.06053504,  0.05512707, -0.00586903,  0.07915846, -0.00263769,
        0.0197233 , -0.07170385, -0.05800329,  0.05786436,  0.10416971,
       -0.13044043, -0.07983699,  0.037913  ,  0.09051442, -0.13

In [13]:
import pandas as pd
df_train = pd.DataFrame([],columns=range(100))
df_test = pd.DataFrame([],columns=range(100))
for i in tqdm(range(len(train_corpus))):
    df_train.loc[len(df_train)] = model.infer_vector(train_corpus[i][0])

for i in tqdm(range(len(test_corpus))):
    df_test.loc[len(df_test)] = model.infer_vector(test_corpus[i][0])

  0%|          | 0/255349 [00:00<?, ?it/s]

  0%|          | 0/85116 [00:00<?, ?it/s]

In [21]:
df_train.to_csv('ian_train.csv',index=False)
df_test.to_csv('ian_test.csv',index=False)

In [22]:
train_df.to_csv('train_df.csv',index=False)
test_df.to_csv('test_df.csv',index=False)

In [14]:
train_df['values'] = train_df['values'].replace(['Y','N'],[1,0])
test_df['values'] = test_df['values'].replace(['Y','N'],[1,0])

In [15]:
X_train = df_train
y_train = train_df['values']

X_test = df_test
y_test = test_df['values']

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [17]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.004529,-0.043733,0.001802,-0.043283,-0.032517,0.059814,-0.057465,-0.002586,-0.013485,-0.030134,...,-0.054937,-0.033316,0.021547,-0.021499,-0.071905,-0.105131,0.011763,-0.005557,0.068240,-0.013557
1,-0.056286,-0.098391,0.034283,-0.085443,0.022977,0.108745,0.072307,-0.072824,0.006076,-0.034458,...,-0.083145,-0.224773,0.213786,-0.084664,0.037299,0.014934,0.158486,0.004323,0.074261,0.116642
2,-0.079125,0.065749,0.037107,-0.034102,-0.025761,0.057848,0.012055,-0.033102,0.038911,0.018438,...,-0.044616,-0.016839,-0.003345,0.006169,0.013835,0.029950,-0.012547,0.004000,0.026409,-0.060217
3,-0.006304,-0.064680,-0.066481,-0.018212,-0.041912,0.039371,0.000956,-0.057000,-0.036842,-0.015193,...,-0.084852,-0.010686,0.038794,0.052108,-0.061928,-0.080209,-0.002736,-0.000235,-0.001798,-0.019634
4,-0.209072,-0.137871,-0.031193,0.070123,-0.082381,0.076852,0.141392,-0.201880,0.092850,-0.007655,...,0.001133,0.064268,0.003482,0.084424,-0.179873,-0.059580,0.089670,0.019427,0.121253,0.007030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255344,0.006350,-0.077087,-0.021659,-0.000186,-0.020764,0.123143,-0.025926,-0.127578,0.058931,-0.025776,...,-0.144684,-0.054785,0.044029,0.114270,-0.066599,-0.107931,-0.020047,0.015422,0.043184,-0.028693
255345,0.015196,0.009674,0.023484,0.011249,-0.000086,0.097884,0.013709,-0.051839,-0.017263,-0.000186,...,-0.035112,-0.011215,0.012756,0.080323,-0.029640,-0.002866,0.007797,0.009470,0.042444,-0.014956
255346,-0.033471,-0.000674,0.017947,0.028989,-0.024082,0.117704,-0.000930,-0.057137,0.069201,0.020728,...,-0.080715,-0.041141,0.105147,0.089262,-0.003091,-0.051274,0.013239,-0.020272,0.108662,-0.017606
255347,-0.074642,-0.087102,0.002657,0.015928,-0.034007,-0.071022,0.019911,-0.069370,-0.017913,-0.014220,...,-0.002577,0.014856,-0.003547,0.086672,-0.048006,-0.019311,-0.030845,-0.043173,0.020691,0.057927


In [18]:
xgb = XGBClassifier(n_jobs=-1,verbose=1)

In [19]:
xgb.fit(X_train,y_train)

C:\Users\avitr\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


AttributeError: module 'pandas' has no attribute 'Int64Index'

In [ ]:
pred = xgb.predict(X_test)

In [ ]:
print(classification_report(y_test,pred))